In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from scipy.interpolate import griddata

In [57]:
# Preprocessing Data (Filter out unwanted columns and combining symptons into a list in merged_symptons)
CSV_Data = './2021VAERSDATA.csv'
CSV_Symptoms = './2021VAERSSYMPTOMS.csv'
CSV_Vax = "./2021VAERSVAX.csv" 

df_d = pd.read_csv(CSV_Data, names = ['VAERS_ID','RECVDATE','STATE','AGE_YRS','CAGE_YR','CAGE_MO','SEX','RPT_DATE','SYMPTOM_TEXT','DIED','DATEDIED','L_THREAT','ER_VISIT','HOSPITAL','HOSPDAYS','X_STAY','DISABLE','RECOVD','VAX_DATE','ONSET_DATE','NUMDAYS','LAB_DATA','V_ADMINBY','V_FUNDBY','OTHER_MEDS','CUR_ILL','HISTORY','PRIOR_VAX','SPLTTYPE','FORM_VERS','TODAYS_DATE','BIRTH_DEFECT','OFC_VISIT','ER_ED_VISIT','ALLERGIES'],error_bad_lines=False, index_col=False, dtype='unicode')
df = df_d.drop(['CAGE_YR', 'CAGE_MO', 'RPT_DATE','SYMPTOM_TEXT','DATEDIED','L_THREAT','ER_VISIT','HOSPITAL','HOSPDAYS','X_STAY','DISABLE','RECOVD','VAX_DATE','ONSET_DATE','NUMDAYS','LAB_DATA','V_ADMINBY','V_FUNDBY','OTHER_MEDS','PRIOR_VAX','SPLTTYPE','FORM_VERS','TODAYS_DATE','BIRTH_DEFECT','OFC_VISIT','ER_ED_VISIT'], axis=1)
df_s = pd.read_csv(CSV_Symptoms, names = ['VAERS_ID','SYMPTOM1','SYMPTOMVERSION1','SYMPTOM2','SYMPTOMVERSION2','SYMPTOM3','SYMPTOMVERSION3','SYMPTOM4','SYMPTOMVERSION4','SYMPTOM5','SYMPTOMVERSION5'])
df_s = df_s.drop(['SYMPTOMVERSION1','SYMPTOMVERSION2','SYMPTOMVERSION3','SYMPTOMVERSION4','SYMPTOMVERSION5'], axis = 1)
df_s['MERGED_SYMPTONS'] = df_s[df_s.columns[1:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
df_s = df_s.drop(['VAERS_ID','SYMPTOM1','SYMPTOM2','SYMPTOM3','SYMPTOM4','SYMPTOM5'], axis = 1)
df = pd.concat([df,df_s], axis=1, join="inner")
df_v = pd.read_csv (CSV_Vax, names = ['VAERS_ID','VAX_TYPE','VAX_MANU','VAX_LOT','VAX_DOSE_SERIES','VAX_ROUTE','VAX_SITE','VAX_NAME'])
df_v = df_v.drop(['VAERS_ID','VAX_TYPE','VAX_LOT','VAX_ROUTE','VAX_SITE','VAX_NAME'], axis = 1)
df = pd.concat([df,df_v], axis=1, join="inner")

df

,VAERS_ID,RECVDATE,STATE,AGE_YRS,SEX,DIED,CUR_ILL,HISTORY,ALLERGIES,MERGED_SYMPTONS,VAX_MANU,VAX_DOSE_SERIES
0,0916600,01/01/2021,TX,33.0,F,NaN,None,None,Pcn and bee venom,"Dysphagia,Epiglottitis",MODERNA,1
1,0916601,01/01/2021,CA,73.0,F,NaN,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,"""Dairy""","Anxiety,Dyspnoea",MODERNA,1
2,0916602,01/01/2021,WA,23.0,F,NaN,None,None,Shellfish,"Chest discomfort,Dysphagia,Pain in extremity,V...",PFIZER\BIONTECH,1
3,0916603,01/01/2021,WA,58.0,F,NaN,kidney infection,"diverticulitis, mitral valve prolapse, osteoar...","Diclofenac, novacaine, lidocaine, pickles, tom...","Dizziness,Fatigue,Mobility decreased",MODERNA,UNK
4,0916604,01/01/2021,TX,47.0,F,NaN,Na,NaN,Na,"Injection site erythema,Injection site pruritu...",MODERNA,1
...,...,...,...,...,...,...,...,...,...,...,...,...
34116,1113917,03/19/2021,FL,88.0,F,NaN,NaN,Medical History/Concurrent Conditions: Aortic ...,NaN,"Full blood count normal,Hepatic cyst,Hepatic l...",MODERNA,2
34117,1113920,03/19/2021,PA,83.0,F,NaN,NaN,Medical History/Concurrent Conditions: Atrial ...,NaN,"Pelvic venous thrombosis,Protein C,Protein S n...",MODERNA,1
34118,1113963,03/19/2021,MS,59.0,F,Y,NaN,Medical History/Concurrent Conditions: Type II...,NaN,"Spleen scan normal,Thrombin-antithrombin III c...",MODERNA,1
34119,1115045,03/19/2021,AZ,57.0,M,Y,NaN,Medical History/Concurrent Conditions: No adve...,NaN,"Dizziness,Hypotension,Myalgia,Nausea,Tremor",MODERNA,1


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34121 entries, 0 to 34120
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   VAERS_ID         34121 non-null  object
 1   RECVDATE         34121 non-null  object
 2   STATE            28550 non-null  object
 3   AGE_YRS          30933 non-null  object
 4   SEX              34121 non-null  object
 5   DIED             1957 non-null   object
 6   CUR_ILL          16069 non-null  object
 7   HISTORY          22375 non-null  object
 8   ALLERGIES        18587 non-null  object
 9   MERGED_SYMPTONS  34121 non-null  object
 10  VAX_MANU         34121 non-null  object
 11  VAX_DOSE_SERIES  34060 non-null  object
dtypes: object(12)
memory usage: 3.1+ MB
